<a href="https://colab.research.google.com/github/jwl25b/Deep-Learning-Final-Project/blob/main/Mini-Dalle/Mini_Dalle_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
prompts = ["Quarter-Zip V-Neck Black Sweatshirt with Tommy Logo at the chest",
    "Solid Crewneck Navy Blue Crewneck polo with signature stripe tipping",
    "Embroidered Tommy Logo Green T-Shirt with textured boxy knit",
    "Slim Fit Stripe Classic White Polo with Tommy Flag Logo"]

In [2]:
import PIL
from PIL import Image
from numpy import asarray


# load the image and convert into
# numpy array
tommy_data = []
dalle_data = []
dalle_sweatshirt_data = []

tommy_path = 'tommy_dataset/'
dalle_path = 'dalle_dataset/'
dalle_sweatshirt_path = 'dalle_sweatshirt_data/'

data_length = 4
sweatshirt_length = 20

for i in range(data_length):
  tommy_data.append(Image.open(tommy_path+'im'+str(i+1)+'.jpg'))
  dalle_data.append(Image.open(dalle_path+'im'+str(i+1)+'.png'))

for i in range(sweatshirt_length):
  dalle_sweatshirt_data.append(Image.open(dalle_sweatshirt_path+'im'+str(i+1)+'.png'))

# asarray() class is used to convert
# PIL images into NumPy arrays
for i in range(0,4):
  tommy_data[i] = asarray(tommy_data[i])
  dalle_data[i] = asarray(dalle_data[i])
tommy_data = asarray(tommy_data)
dalle_data = asarray (dalle_data)

for i in range(0,10):
  dalle_sweatshirt_data[i] = asarray(dalle_sweatshirt_data[i])

tommy_data = asarray(tommy_data)
dalle_data = asarray (dalle_data)
dalle_sweatshirt_data = asarray(dalle_sweatshirt_data)

<ipython-input-2-140276df556c>:39: FutureWarning: The input object of type 'WebPImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'WebPImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  dalle_sweatshirt_data = asarray(dalle_sweatshirt_data)
<ipython-input-2-140276df556c>:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dalle_sweatshirt_data = asarray(dalle_sweatshirt_data)


In [3]:
import torch
import numpy as np
from torchvision.transforms import functional as F

def preprocess_image(image):
    image = np.array(image).astype(np.float32)
    image = torch.tensor(image).unsqueeze(0)
    image = image.permute(0, 3, 1, 2) / 255.0
    return F.center_crop(image, (512, 512))

In [4]:
asarray(preprocess_image(tommy_data[0]))
print(asarray(preprocess_image(tommy_data[0])).shape)
asarray(preprocess_image(dalle_data[0]))
print(asarray(preprocess_image(dalle_data[0])).shape)

(1, 3, 512, 512)
(1, 3, 512, 512)


In [5]:
!pip install torchmetrics

In [6]:
images_np = np.concatenate([preprocess_image(image) for image in dalle_data])
#dalle_sweatshirt_data2 = dalle_sweatshirt_data.repeat(2, axis=0)
dalle_sweatshirt_np = np.concatenate([preprocess_image(image) for image in dalle_sweatshirt_data])
print(dalle_sweatshirt_data.shape)

(20,)


In [7]:
from torchmetrics.functional.multimodal import clip_score
from functools import partial

clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")


def calculate_clip_score(images_np, prompts):
    images_int = (images_np * 255).astype("uint8")
    clip_score = clip_score_fn(torch.from_numpy(images_int), prompts).detach()
    return round(float(clip_score), 4)


sd_clip_score = calculate_clip_score(images_np, prompts)

In [8]:
sd_clip_score

25.4034

In [9]:
!pip install torchmetrics[image]
!pip install torch-fidelity

In [10]:
import torch
import torchmetrics
import torch_fidelity
from torchmetrics.image.inception import InceptionScore

def calculate_inception_score(images):
    images_int = (images * 255).astype("uint8")
    images_int = torch.from_numpy(images_int).detach()
    metric = InceptionScore()
    metric.update(images_int)
    return metric


sd_inception_score = calculate_inception_score(dalle_sweatshirt_np)

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [11]:
print(f"Inception Score: {float(sd_inception_score.compute()[0].item())}")

Inception Score: 1.217350721359253


In [12]:
tommy_images = torch.cat([preprocess_image(image) for image in tommy_data])
print(tommy_images.shape)
dalle_images = torch.cat([preprocess_image(image) for image in dalle_data])
print(tommy_images.shape)

torch.Size([4, 3, 512, 512])
torch.Size([4, 3, 512, 512])


In [13]:
from torchmetrics.image.fid import FrechetInceptionDistance

fid = FrechetInceptionDistance(normalize=True)
fid.update(tommy_images, real=True)
fid.update(dalle_images, real=False)

In [14]:
print(f"FID: {float(fid.compute())}")

FID: 354.60272216796875
